# Langchain learning
https://www.freecodecamp.org/news/beginners-guide-to-langchain/

In [ ]:
!pip install langchain_core langchain_anthropic

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# os.environ['LANGCHAIN_API_KEY']
# os.environ['TAVILY_API_KEY']
# os.environ['ANTHROPIC_API_KEY']

# Local CRAG LangGraph

In [ ]:
## Ollama
!ollama pull mistral:instruct

In [ ]:
local_llm = "mistral:instruct"

# Index

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings

# Load
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(url)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

# Embed and index
embedding = GPT4AllEmbeddings()

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding
)
retriever = vectorstore.as_retriever()

# JSON Mode

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n
    Here is the retrieved document: \n\n {context} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=['question', 'context'],
)

chain = prompt | llm | JsonOutputParser()
question = "Explain how the different types of agent memory work?"
docs = retriever.get_relevant_documents(question)
score = chain.invoke({"question": question, "context": docs[0].page_content})
"{'score': 'yes'}"

# Graph State

In [ ]:
from typing import Annotated, Dict, TypedDict
from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [ ]:
import json
import operator
from typing import Annotated, Dict, TypedDict

from langchain import hub
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai.chat_models import ChatMistralAI


### Nodes ###

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state['keys']
    question = state_dict['question']
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "question": question}}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains generation
    """
    print("---GENERATE---")
    state_dict = state['keys']
    question = state_dict['question']
    documents = state_dict['documents']

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")
    # prompt = PromptTemplate(
    #     template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n
    #     Question: {question}\n
    #     Context: {context}\n
    #     Answer:""",
    #     input_variables=['question', 'context'],
    # )

    # LLM
    llm = ChatOllama(model=local_llm, temperature=0)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {
            "documents": documents,
            "question": question,
            "generation": generation
        }
    }


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant document
    """

    print("---CHECK RELEVANCE---")
    state_dict = state['keys']
    question = state_dict['question']
    documents = state_dict['documents']

    # LLM
    llm = ChatOllama(model=local_llm, format="json", temperature=0)

    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question \n
        Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
        input_variables=['question', 'context'],
    )

    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    search = "No"
    for d in documents:
        score = chain.invoke({"question": question, "context": d.page_content})
        grade = score['score']
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search = "Yes"  # Perform web search
            continue

    return {
        "keys": {
            "documents": filtered_docs,
            "question": question,
            "run_web_search": search,
        }
    }


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    state_dict = state['keys']
    question = state_dict['question']
    documents = state_dict['documents']

    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n
        Look at the input and try to reason about the underlying sematic intent / meaning. \n
        Here is the initial question:
        \n --------\n
        {question}
        \n --------\n
        Provide an improved question without any preamble, only respond with the updated question""",
        input_variables=["question"],
    )

    # LLM
    llm = ChatOllama(model=local_llm, temperature=0)

    # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {
        "keys": {
            "documents": documents,
            "question": better_question
        }
    }


def web_search(state):
    """
    Web search based on the re-phrased question using Tavily API.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Web results appended to documents.
    """
    print("---WEB SEARCH---")
    state_dict = state['keys']
    question = state_dict['question']
    documents = state_dict['documents']

    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})
    web_results = "\n".join([d['content'] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {
        "keys": {
            "documents": documents,
            "question": question
        }
    }


### Edges

def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Next node to call
    """
    print("---WEB SEARCH---")
    state_dict = state['keys']
    question = state_dict['question']
    filtered_documents = state_dict['documents']
    search = state_dict['run_web_search']

    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY AND RUN WEB SEARCH---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [ ]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search", web_search)

# Build Graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    }
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [ ]:
# Run
inputs = {
    "keys": {
        # "question": "Explain how the different types of agent memory work?",
        "question": "Explain how AlphaCodium works?",
    }
}

for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value['keys']['generation'])
